In [24]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

# Define Table which needs to be switched
List all tables in databricks which needs to be switch, because they have the column name account and account_id

In [32]:
from datetime import datetime
import pandas as pd
import delta_sharing
import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids

In [26]:
profile_path = db_client.return_databricks_client()
client = delta_sharing.SharingClient(profile_path)
tables = client.list_all_tables()
print(f"Number of tables: {len(tables)}")
columns_to_replace = ["account", "account_id"]
result = pd.DataFrame(columns=["share", "schema", "name", "account", "account_id"])

In [27]:
tables = client.list_all_tables()
print(tables)


In [30]:
for itable, table in enumerate(tables):
    print(f"{itable}: Table name: {table.name}")
    if table.name in result["name"].unique():
        print(f".  Table name already in result: {table.name}")
        continue
    row = len(result)
    result.loc[row,:] = [table.share, table.schema, table.name, False, False]
    table_path = f"{profile_path}#{table.share}.{table.schema}.{table.name}"
    temp = delta_sharing.load_as_pandas(table_path, limit=1)
    for column in columns_to_replace:
        if column in temp.columns:
            result.at[row, column] = True

In [33]:
result.to_csv(f"/Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/SprintStories/Migrate-account-to-workspace-id/list-tables-with-variables-to-switch-{datetime.now()}.csv", index=False)

In [34]:
result_filtered = result[result["account"] | result["account_id"]]
result_filtered

In [35]:
result_filtered[result_filtered["schema"]=="monitoring"]["name"].unique()